In [ ]:
# import polars as pl
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader, random_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error, r2_score
# import os
# import numpy as np
# from tqdm import tqdm

# # Check for GPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")


# # 1. Define the Dataset Class
# class PolarsDataset(Dataset):
#     def __init__(self, X_files, y_files, scaler=None):
#         self.X_files = X_files
#         self.y_files = y_files
#         self.scaler = scaler or StandardScaler()
#         self.fitted = False  # Track if the scaler is fitted

#     def __len__(self):
#         return len(self.X_files)

#     def __getitem__(self, idx):
#         # Load a single batch
#         X_batch = pl.read_parquet(self.X_files[idx]).to_pandas()
#         y_batch = pl.read_parquet(self.y_files[idx]).to_pandas().squeeze()
        
#         # Standardize features
#         if not self.fitted:
#             X_batch = self.scaler.fit_transform(X_batch)
#             self.fitted = True
#         else:
#             X_batch = self.scaler.transform(X_batch)
        
#         X_tensor = torch.tensor(X_batch, dtype=torch.float32)
#         y_tensor = torch.tensor(y_batch.values, dtype=torch.float32).unsqueeze(1)
        
#         return X_tensor, y_tensor


# # 2. Define the PyTorch Model
# class SimpleMLP(nn.Module):
#     def __init__(self, input_size):
#         super(SimpleMLP, self).__init__()
#         self.hidden1 = nn.Linear(input_size, 10)
#         self.hidden2 = nn.Linear(10, 5)
#         self.output = nn.Linear(5, 1)
#         self.relu = nn.ReLU()
#         self.bn1 = nn.BatchNorm1d(10)
#         self.dropout = nn.Dropout(0.2)
#         self.bn2 = nn.BatchNorm1d(5)
#         self.bn3 = nn.BatchNorm1d(1)
        
#     def forward(self, x):
#         x = self.relu(self.hidden1(x))
#         x = self.bn1(x)
#         x = self.dropout(x)
#         x = self.relu(self.hidden2(x))
#         x = self.bn2(x)
#         x = self.dropout(x)
#         return self.output(x)


# # 3. List all batch files
# X_files = sorted([f"../processed_data/features/{f}" for f in os.listdir('../processed_data/features') if f.endswith('.parquet')])
# y_files = sorted([f"../processed_data/lables/{f}" for f in os.listdir('../processed_data/lables') if f.endswith('.parquet')])
# assert len(X_files) == len(y_files), "Mismatch between X and y batch files"


# # 4. Initialize Dataset and Split into Train/Test
# scaler = StandardScaler()
# dataset = PolarsDataset(X_files, y_files, scaler)

# train_size = int(0.8 * len(dataset))
# test_size = len(dataset) - train_size
# train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)


# # 5. Initialize Model, Loss, and Optimizer
# input_size = pl.read_parquet(X_files[0]).shape[1]
# model = SimpleMLP(input_size).to(device)
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.3)


# # 6. Training Loop with Multiple Epochs
# EPOCHS = 20  # Number of epochs

# for epoch in range(EPOCHS):
#     model.train()
#     epoch_loss = 0.0
#     all_preds = []
#     all_targets = []
    
#     for batch_idx, (X_batch, y_batch) in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{EPOCHS}", unit="batch")):
#         # Squeeze out the extra dimension
#         X_batch = X_batch.squeeze(0)
#         y_batch = y_batch.squeeze(0)
#         X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
#         # Forward Pass
#         optimizer.zero_grad()
#         outputs = model(X_batch)
#         loss = criterion(outputs, y_batch)
            
#         # Backward Pass
#         loss.backward()
#         optimizer.step()
        
#         epoch_loss += loss.item()
        
#         # Store predictions and targets for R² score
#         preds = outputs.detach().cpu().numpy().reshape(-1, 1)
#         targs = y_batch.detach().cpu().numpy().reshape(-1, 1)
#         all_preds.append(preds)
#         all_targets.append(targs)
    
#     # Calculate average loss and R² score
#     avg_loss = epoch_loss / len(train_dataloader)
#     all_preds = np.vstack(all_preds)
#     all_targets = np.vstack(all_targets)
#     r2 = r2_score(all_targets, all_preds)
    
#     print(f"Epoch {epoch+1}/{EPOCHS} - Average Loss: {avg_loss:.4f} - R² Score: {r2:.4f}")


# # 7. Evaluation on Test Set
# model.eval()
# all_preds = []
# all_targets = []

# with torch.no_grad():
#     for X_batch, y_batch in tqdm(test_dataloader, desc="Evaluating on Test Set", unit="batch"):
#         X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#         outputs = model(X_batch)
        
#         preds = outputs.cpu().numpy().reshape(-1, 1)
#         targs = y_batch.cpu().numpy().reshape(-1, 1)
#         all_preds.append(preds)
#         all_targets.append(targs)

# # Aggregate Predictions and Targets
# all_preds = np.vstack(all_preds)
# all_targets = np.vstack(all_targets)

# # Calculate Final Test Metrics
# mse = mean_squared_error(all_targets, all_preds)
# r2 = r2_score(all_targets, all_preds)

# print("\nTraining Complete!")
# print(f"📊 Final Test MSE: {mse:.4f}")
# print(f"📈 Final Test R² Score: {r2:.4f}")